# The Noon-Shadow Angle

TODO words


In [1]:
import sympy as sp
from sympy import sin
import numpy as np
from analemma.algebra import frame, render, result

e1, e2, e3 = frame.base("e")
f1, f2, f3 = frame.planet()
n1, n2, n3 = frame.surface()
M = frame.meridian_plane() # n1^n3
s = frame.sunray()

m1, m2, m3 = frame.dial()
gn = frame.gnomon("n", zero_decl=True)
g = frame.gnomon("e", zero_decl=True)

## The Hour Angle

TODO recap hour angle and words about simplifying the shadow bivector, Note that sin(Xi) is overall factor.

TODO use words from here

The calculation proceeds by defining $S = s \wedge g$, the plane containing the sunshine vector and the gnomon. Its magnitude is the (negative) cosine of the angle between $s$ and $g$. Then we can obtain a generalized solar hour angle $\mu$ as the angle between $S$ and $M$, given by

$$\cos(\mu) = \frac{-S\cdot M}{\sqrt{-S^2}\sqrt{-M^2}} = \frac{-S\cdot M}{\sin(\Xi)}$$

given that $M^2=-1$. In anticipation of setting $\delta =0$, I will apply $\delta$ as a subscript to these quantities for now.

In [2]:
S = result.shadow_bivector(s, g)
Sn = result.shadow_bivector_explicit()

render.simplification("S", [S, Sn], as_lines=[True, False])

<IPython.core.display.Math object>

where the hour angle $\tan(\mu)$ is defined by

In [3]:
render.expression(r"\tan(\mu)", result.hour_angle_tan(S, M))

<IPython.core.display.Math object>

In [4]:
render.expression(r"\cos(\Xi)", result.shadow_bivector_magnitude_angle_cos(s, g))

<IPython.core.display.Math object>

## The Unit Shadow

Form a vector $u$ parallel to the shadow, as the intersection between $S$ (the plane containing the sun ray and the gnomon) and $G$ (the dial face) where

$$u = \langle I G S \rangle_1$$

TODO link to cheat sheet

In [5]:
Gn = frame.dialface()
u = result.dialface_shadowbivector_intersection(Gn, Sn)
render.align("u", u)

<IPython.core.display.Math object>

Want to normalize $u$ to get $\hat{u} \equiv \hat{w}$, a unit vector parallel to the shadow $w$. Can square and add the above components, but there is a route to a simpler expression as follows.

The cosine of the angle between the "shadow plane" $S$ (containing the sun ray $s$ and the gnomon $g$) and dial face $G$ is given by

$$\cos(\Psi) = \frac{S\cdot G}{\sqrt{-S^2}\sqrt{-G^2}}$$

Given that $S^2 = -\sin^2(\Xi)$ and $G^2 = -1$,

$$ \cos(\Psi) = \frac{S\cdot G}{\sin(\Xi)}$$

We can verify the equivalence of these two forms of $\cos(\Psi)$ as follows,

In [6]:
# check (force sympy to give up the positive square root in each factor in the denominator)
def _norm(B):
    return sp.powdenest(sp.sqrt(sp.trigsimp((-B|B).obj)), force=True)

Xi = sp.Symbol(r"\Xi")
SuGu_check = sp.trigsimp((Sn|Gn).obj)/_norm(Sn)/_norm(Gn) - (Sn|Gn)/sin(Xi)
assert SuGu_check.obj.equals(0)
display( SuGu_check )

0

and calculate a simple expression for $\cos(\Psi)$:

In [7]:
cosPsi = result.dialface_shadowbivector_angle_cos(Gn, Sn)
render.expression(r"\cos(\Psi)", cosPsi)

<IPython.core.display.Math object>

Now, the length of $u$ is related to $\Psi$ as follows.

$$u^2 = (I\; G \times S)^2 = (\frac{I}{2}(GS-SG))^2 = -\frac{1}{4}(GSGS+SGSG-2S^2G^2)$$

But

$$(S\cdot G)^2 = \frac{1}{4}(SG+GS)^2 = \frac{1}{4}(SGSG+GSGS+2S^2G^2)$$

So,

$$u^2 = S^2G^2 - (S\cdot G)^2 = \sin^2(\Xi) - (S\cdot G)^2 = \sin^2(\Xi)\sin^2(\Psi)$$

TODO link to cheat sheet

In [8]:
# check TODO consider moving to a test file for speed
u2_check = (u|u) - (sin(Xi)**2 - (Sn|Gn)**2)
assert u2_check.obj.equals(0)
display( u2_check )

0

We can now form $\hat{w}$ by dividing $u$ by its length

TODO explain the $u$ has some arbitrary length

In [9]:
what = result.unit_shadow(Gn, Sn)

render.align(r"\hat{w}", what) # TODO factor (in update_coeffs for now) also basis vectors are zero-based


<IPython.core.display.Math object>

Check that

$$\hat{w}^2 = \left(\frac{u}{\sin(\Xi)\sin(\Psi)}\right)^2 = 1$$

by showing that

$$\frac{u^2}{\sin^2(\Xi)} - \sin^2\Psi = 0$$

TODO move to test file so runs in parallel

In [10]:
sinPsiSquared = 1 - sp.expand(cosPsi**2)
u_over_sinXi_squared = sp.trigsimp(((u|u)/sin(Xi)**2).obj)
what_check = sp.trigsimp(u_over_sinXi_squared - sinPsiSquared)
assert what_check.equals(0)
display( what_check )

0

## The Shadow at Noon

Get the angular coordinate of the shadow tip relative to noon, call it $\zeta$. At noon we have $\mu = 0$

In [11]:
render.expression(r"\hat{w}_{\mu=0}", result.unit_noon_shadow(Gn, Sn))

<IPython.core.display.Math object>

So $\cos(\zeta) = \hat{w}(\mu) \cdot \hat{w}(0)$ and the easiest route to $\sin(\zeta)$ is to use the fact that the shadow lives in the plane of the dial face, so $\hat{w}(\mu) \wedge \hat{w}(0) = \sin(\zeta) G$.

In [12]:
render.expressions(
    (r"\sin(\zeta)", r"\cos(\zeta)", r"\tan(\zeta)"),
    result.noon_angle_sincos(Gn, Sn) + (result.noon_angle_tan(Gn, Sn),)
    )

<IPython.core.display.Math object>

In [13]:
# TODO move tests from sundial.ipynb to tests/